In [30]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [31]:
ROOT_DIR = '/content/gdrive/My Drive/Yolov8Developer/Yolov8Dataset'

In [32]:
%pip install ultralytics

In [33]:
import os

from ultralytics import YOLO
import pandas as pd
import cv2

In [34]:
VIDEOS_DIR = os.path.join('/content/gdrive/MyDrive/Yolov8Developer/Yolov8Dataset/datasets/coco/videos')

In [35]:
video_path = os.path.join(VIDEOS_DIR, 'daytime42sec.mp4')
video_path_out = '{}_out.mp4'.format(video_path)

In [36]:
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
H, W, _ = frame.shape
out = cv2.VideoWriter(video_path_out, cv2.VideoWriter_fourcc(*'MP4V'), int(cap.get(cv2.CAP_PROP_FPS)), (W, H))

In [37]:
model_path = os.path.join('/content/gdrive/MyDrive/Yolov8Developer/Yolov8Dataset/runs/detect/20231105epoch20/weights/last.pt')

In [38]:
# Load a model
model = YOLO(model_path)  # load a custom model

# Data for Excel
data = {'Frame': [], 'Label': [], 'Confidence': []}

# Summary data for each label
label_summary = {'private passenger car': 0, 'business passenger car': 0, 'bus': 0, 'motorcycle': 0}

# Set to keep track of processed frames
processed_frames = set()

threshold = 0.5

while ret:
    # 將每個 frame 的標籤數歸零
    frame_label_summary = {'private passenger car': 0, 'business passenger car': 0, 'bus': 0, 'motorcycle': 0}

    results = model(frame)[0]

    frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

    # Skip frames that have been processed already
    if frame_number in processed_frames:
      ret, frame = cap.read()
      continue

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, confidence, class_id = result

        if confidence > threshold:
            label = model.names[int(class_id)].lower()

            # Append data
            data['Frame'].append(int(cap.get(cv2.CAP_PROP_POS_FRAMES)))
            data['Label'].append(label)
            data['Confidence'].append(confidence)

            # Update label summary count (convert to lowercase)
            label_summary[label] += 1

            # Draw bounding box and label on the frame
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
            cv2.putText(frame, label, (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)
    # 更新整體標籤總數
    for label, count in frame_label_summary.items():
        label_summary[label] += count

    # 標記已處理的幀
    processed_frames.add(frame_number)

    # Write frame to output video
    out.write(frame)
    ret, frame = cap.read()

# Release video capture and writer
cap.release()
out.release()
cv2.destroyAllWindows()

# Create DataFrame from data
df = pd.DataFrame(data)

# Summary DataFrame
summary_df = pd.DataFrame(list(label_summary.items()), columns=['Label', 'Count'])

# Group by label and count occurrences
label_counts = df['Label'].value_counts()

# Calculate precision, recall, and F1 score for each label
precision_recall_f1 = {'Label': [], 'Precision': [], 'Recall': [], 'F1 Score': []}
for label in label_counts.index:
    true_positive = df[(df['Label'] == label) & (df['Confidence'] > threshold)].shape[0]
    false_positive = df[(df['Label'] != label) & (df['Confidence'] > threshold)].shape[0]
    false_negative = label_counts[label] - true_positive

    precision = true_positive / (true_positive + false_positive + 1e-5)
    recall = true_positive / (true_positive + false_negative + 1e-5)

    # Calculate F1 score
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-5)

    precision_recall_f1['Label'].append(label)
    precision_recall_f1['Precision'].append(precision)
    precision_recall_f1['Recall'].append(recall)
    precision_recall_f1['F1 Score'].append(f1_score)

# Save DataFrame to Excel
output_path = '/content/gdrive/MyDrive/Yolov8Developer/Yolov8Dataset/datasets/coco/videos/daytime42sec_results.xlsx'
with pd.ExcelWriter(output_path) as writer:
  df.to_excel(writer, sheet_name='Detection Results', index=False)
  summary_df.to_excel(writer, sheet_name='Label Summary', index=False)
  pd.DataFrame(precision_recall_f1).to_excel(writer, sheet_name='Precision and Recall', index=False)


0: 384x640 2 Business Passenger Cars, 5 Private Passenger Cars, 1 Motorcycle, 693.2ms
Speed: 7.7ms preprocess, 693.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Business Passenger Car, 5 Private Passenger Cars, 1 Motorcycle, 300.3ms
Speed: 11.3ms preprocess, 300.3ms inference, 5.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Business Passenger Car, 5 Private Passenger Cars, 1 Motorcycle, 230.4ms
Speed: 4.9ms preprocess, 230.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Business Passenger Car, 4 Private Passenger Cars, 1 Motorcycle, 448.6ms
Speed: 5.0ms preprocess, 448.6ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Business Passenger Cars, 4 Private Passenger Cars, 1 Motorcycle, 229.8ms
Speed: 4.3ms preprocess, 229.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Business Passenger Car, 3 Private Passenger Cars, 1 Motorcycle,